# --- `Day 19`: Beacon Scanner ---

In [79]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations, permutations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [11]:
def parse_line(line): 
    return line.splitlines()
    
def parse_input(input):
    return list(map(parse_line, input.split("\n\n")))

In [ ]:
final_input = parse_input(aocd.get_data(day=19, year=2021))
print(final_input[:5])

In [13]:
test_input = parse_input('''\
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
''')

print(test_input)

[['--- scanner 0 ---', '404,-588,-901', '528,-643,409', '-838,591,734', '390,-675,-793', '-537,-823,-458', '-485,-357,347', '-345,-311,381', '-661,-816,-575', '-876,649,763', '-618,-824,-621', '553,345,-567', '474,580,667', '-447,-329,318', '-584,868,-557', '544,-627,-890', '564,392,-477', '455,729,728', '-892,524,684', '-689,845,-530', '423,-701,434', '7,-33,-71', '630,319,-379', '443,580,662', '-789,900,-551', '459,-707,401'], ['--- scanner 1 ---', '686,422,578', '605,423,415', '515,917,-361', '-336,658,858', '95,138,22', '-476,619,847', '-340,-569,-846', '567,-361,727', '-460,603,-452', '669,-402,600', '729,430,532', '-500,-761,534', '-322,571,750', '-466,-666,-811', '-429,-592,574', '-355,545,-477', '703,-491,-529', '-328,-685,520', '413,935,-424', '-391,539,-444', '586,-435,557', '-364,-763,-893', '807,-499,-711', '755,-354,-619', '553,889,-390'], ['--- scanner 2 ---', '649,640,665', '682,-795,504', '-784,533,-524', '-644,584,-595', '-588,-843,648', '-30,6,44', '-674,560,763', '50

### Helpers

In [173]:
list(permutations([0, 1, 2]))

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [204]:
def parse(input):
    scanners = {}
    for i,line in enumerate(input):
        scans = set()
        for data in line:
            if "scanner" not in data:
                scan = tuple(ints(data))
                scans.add(scan)
        scanners[i] = scans
    return scanners

def get6(point):
    result = []
    for a,b,c in permutations([0, 1, 2]):
        result.append((point[a], point[b], point[c]))
    return result

def rotate(point, n):
    master = get6(point)
    
    if 0 <= n < 6:
        return master[n]
    elif 6 <= n < 12:
        return (-master[n - 6][0], master[n - 6][1], master[n - 6][2])
    elif 12 <= n < 18:
        return (master[n - 12][0], -master[n - 12][1], master[n - 12][2])
    elif 18 <= n < 24:
        return (master[n - 18][0], master[n - 18][1], -master[n - 18][2])
    elif 24 <= n < 30:
        return (-master[n - 30][0], -master[n - 30][1], master[n - 30][2])
    elif 30 <= n < 36:
        return (-master[n - 36][0], master[n - 36][1], -master[n - 36][2])
    elif 36 <= n < 42:
        return (master[n - 42][0], -master[n - 42][1], -master[n - 42][2])
    elif 42 <= n < 48:
        return (-master[n - 48][0], -master[n - 48][1], -master[n - 48][2])

def countOverlap(s1, s2, offset):
    dx,dy,dz = offset
    copy = {(x + dx,y + dy,z + dz) for x,y,z in s2}
    return s1.intersection(copy)

def check(found, target):
    for n in range(48):
        modded = {rotate(i,n) for i in target}
        for mod in modded:
            for j in found:
                dx = j[0] - mod[0]
                dy = j[1] - mod[1]
                dz = j[2] - mod[2]

                overlap = countOverlap(found, modded, (dx, dy, dz))
                if len(overlap) >= 12:
                    return dx, dy, dz, modded
    return None,None,None,None

for i in range(48):
    print(rotate((1,2,3), i))

(1, 2, 3)
(1, 3, 2)
(2, 1, 3)
(2, 3, 1)
(3, 1, 2)
(3, 2, 1)
(-1, 2, 3)
(-1, 3, 2)
(-2, 1, 3)
(-2, 3, 1)
(-3, 1, 2)
(-3, 2, 1)
(1, -2, 3)
(1, -3, 2)
(2, -1, 3)
(2, -3, 1)
(3, -1, 2)
(3, -2, 1)
(1, 2, -3)
(1, 3, -2)
(2, 1, -3)
(2, 3, -1)
(3, 1, -2)
(3, 2, -1)
(-1, -2, 3)
(-1, -3, 2)
(-2, -1, 3)
(-2, -3, 1)
(-3, -1, 2)
(-3, -2, 1)
(-1, 2, -3)
(-1, 3, -2)
(-2, 1, -3)
(-2, 3, -1)
(-3, 1, -2)
(-3, 2, -1)
(1, -2, -3)
(1, -3, -2)
(2, -1, -3)
(2, -3, -1)
(3, -1, -2)
(3, -2, -1)
(-1, -2, -3)
(-1, -3, -2)
(-2, -1, -3)
(-2, -3, -1)
(-3, -1, -2)
(-3, -2, -1)


## Solution 1

In [205]:
def solve_1(input):
    scanners = parse(input)
        
    todo = deque()
    for i in range(1, len(scanners)):
        todo.append(i)

    found = set(scanners[0])
    while todo:
        i = todo.popleft()
        dx, dy, dz, modded = check(found, scanners[i])
        if dx != None:
            print("scanner:", i, dx, dy, dz)
            for n in modded:
                found.add((n[0] + dx, n[1] + dy, n[2] + dz))
        else:
            todo.append(i)
                
    return len(found)
          
solve_1(test_input)

scanner: 1 68 -1246 -43
scanner: 3 -92 -2380 -20
scanner: 4 -20 -1133 1061
scanner: 2 1105 -1205 1229


79

In [ ]:
f"Solution 1: {solve_1(final_input)}"

## Solution 2

In [198]:
def solve_2(input):
    scanners = parse(input)
    
    todo = deque()
    for i in range(1, len(scanners)):
        todo.append(i)

    positions = [None] * len(scanners)
    positions[0] = (0, 0, 0)
    found = set(scanners[0])
    while todo:
        i = todo.popleft()
        dx, dy, dz, modded = check(found, scanners[i])
        if dx != None:
            print("scanner:", i, dx, dy, dz)
            for n in modded:
                found.add((n[0] + dx, n[1] + dy, n[2] + dz))
                positions[i] = (dx, dy, dz)
        else:
            todo.append(i)
            
    best = 0
    for a in positions:
        for b in positions:
            distance = abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])
            if distance > best:
                best = distance
    return best
    
solve_2(test_input)

scanner: 1 68 -1246 -43
scanner: 3 -92 -2380 -20
scanner: 4 -20 -1133 1061
scanner: 2 1105 -1205 1229


3621

In [ ]:
f"Solution 2: {solve_2(final_input)}"